In [2]:
Packages <- c("vegan","dplyr", "broom", "fields","geosphere", "data.table",  "ggplot2","stringr",  "lubridate", "RColorBrewer", "viridis")
invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0

In [61]:
relatedness <- fread("empirical_data/genetics/out.noL4437.relatedness2")[,`:=`(INDV1=gsub("APCL_", "", INDV1), INDV2=gsub("APCL_", "", INDV2))]
fish_meta <-readRDS("empirical_data/genetics/fish_meta_latlon.rds")
setDT(fish_meta)
relatedness_meta_int <- left_join(relatedness, fish_meta[, .(ligation_id, INDV1_lat=lat, INDV1_lon=lon)], by=c(INDV1="ligation_id"))
relatedness_meta <- left_join(relatedness_meta_int, fish_meta[, .(ligation_id, INDV2_lat=lat, INDV2_lon=lon)], by=c(INDV2="ligation_id"))[INDV1 != INDV2]

In [93]:
nrow(relatedness_meta)
relatedness_meta$dist_km <- distHaversine(as.matrix(relatedness_meta[,c('INDV1_lon', 'INDV1_lat')]), as.matrix(relatedness_meta[,c('INDV2_lon', 'INDV2_lat')]))/1000
#some fish are super close to each other, as long as identical fish are filtered out of relatedness_meta, swap zeros for really tiny numbers instead
relatedness_meta <- relatedness_meta[, dist_km := ifelse(dist_km==0.000000000, 0.000000001, dist_km)][INDV1 != INDV2]
UnqRelatedness <- unique(relatedness_meta, by=c("RELATEDNESS_PHI", "dist_km"))[, phi := RELATEDNESS_PHI][INDV1 != INDV2]
nrow(UnqRelatedness)

[1] 2308880

[1] 1152438

In [79]:
head(relatedness_meta[INDV1 == INDV2])

INDV1,INDV2,N_AaAa,N_AAaa,N1_Aa,N2_Aa,RELATEDNESS_PHI,INDV1_lat,INDV1_lon,INDV2_lat,INDV2_lon,dist_km


In [82]:
head(relatedness_meta[INDV1 != INDV2, .(INDV1, INDV2, RELATEDNESS_PHI)])[INDV1 == INDV2]

INDV1,INDV2,RELATEDNESS_PHI


In [97]:
#make a matrix of pairwise relatedness and pairwise distance, test for significant correlation
RelatednessMat <- as.matrix(dcast(relatedness_meta[INDV1 != INDV2, .(INDV1, INDV2, RELATEDNESS_PHI)], INDV1~INDV2, value.var = "RELATEDNESS_PHI", drop = TRUE)[, -"INDV1"])
DistMat <- as.matrix(dcast(relatedness_meta[INDV1 != INDV2, .(INDV1, INDV2, dist_km)], INDV1~INDV2, value.var = "dist_km", drop = TRUE)[, -"INDV1"])


In [98]:
head(RelatednessMat)
head(DistMat)

L0255,L0256,L0257,L0258,L0259,L0260,L0261,L0262,L0263,L0264,...,L5609,L5611,L5614,L5621,L5622,L5637,L5638,L5639,L5640,L5655
NA,0.216524,0.152207,0.145374,0.195846,0.184971,0.148629,0.1579710,0.214493,0.171388,...,0.169837,0.148784,0.159463,0.185290,0.168303,0.1735420,0.1368120,0.1518250,0.1125730,0.192572
0.216524,NA,0.134021,0.172119,0.206897,0.191877,0.201399,0.1671350,0.221910,0.206044,...,0.192612,0.169209,0.157287,0.178326,0.176871,0.1958620,0.1272230,0.1810470,0.1359770,0.201602
0.152207,0.134021,NA,0.161094,0.118280,0.139013,0.131343,0.0929535,0.149925,0.159590,...,0.171108,0.118343,0.143519,0.130117,0.118841,0.0897059,0.1078720,0.0936556,0.0816944,0.169034
0.145374,0.172119,0.161094,NA,0.158519,0.165945,0.157061,0.1027500,0.159190,0.195191,...,0.173677,0.110000,0.181548,0.101695,0.103641,0.0980114,0.0873239,0.0962099,0.0861314,0.145604
0.195846,0.206897,0.118280,0.158519,NA,0.196793,0.174672,0.1038010,0.192982,0.188571,...,0.158904,0.145743,0.163910,0.161198,0.169731,0.1908180,0.1180650,0.1605300,0.1179940,0.163662
0.184971,0.191877,0.139013,0.165945,0.196793,NA,0.171631,0.1410260,0.216524,0.168524,...,0.216578,0.147679,0.166911,0.207232,0.182069,0.1790210,0.1442440,0.1936870,0.1149430,0.190798


L0255,L0256,L0257,L0258,L0259,L0260,L0261,L0262,L0263,L0264,...,L5609,L5611,L5614,L5621,L5622,L5637,L5638,L5639,L5640,L5655
NA,0.000000001,0.008412914,0.008412914,0.002973002,0.002554477,0.002554477,0.009765633,0.02064368,0.02064368,...,25.85351,25.86088,27.69636,27.73454,27.74237,27.68494,9.113224,9.113224,13.98158,0.3034665
0.000000001,NA,0.008412914,0.008412914,0.002973002,0.002554477,0.002554477,0.009765633,0.02064368,0.02064368,...,25.85351,25.86088,27.69636,27.73454,27.74237,27.68494,9.113224,9.113224,13.98158,0.3034665
0.008412914,0.008412914,NA,0.000000001,0.006809954,0.010736814,0.010736814,0.018172210,0.02889877,0.02889877,...,25.86023,25.86760,27.70283,27.74100,27.74883,27.69141,9.121340,9.121340,13.98945,0.3116231
0.008412914,0.008412914,0.000000001,NA,0.006809954,0.010736814,0.010736814,0.018172210,0.02889877,0.02889877,...,25.86023,25.86760,27.70283,27.74100,27.74883,27.69141,9.121340,9.121340,13.98945,0.3116231
0.002973002,0.002973002,0.006809954,0.006809954,NA,0.004355045,0.004355045,0.011850753,0.02221500,0.02221500,...,25.85375,25.86112,27.69646,27.73464,27.74247,27.68504,9.114545,9.114545,13.98264,0.3048402
0.002554477,0.002554477,0.010736814,0.010736814,0.004355045,NA,0.000000001,0.007525617,0.01818498,0.01818498,...,25.85099,25.85836,27.69386,27.73204,27.73987,27.68244,9.110732,9.110732,13.97904,0.3009837


In [99]:
dim(RelatednessMat)
dim(DistMat)

[1] 1520 1520

[1] 1520 1520

In [100]:
mantel(RelatednessMat, DistMat, na.rm = TRUE)


Mantel statistic based on Pearson's product-moment correlation 

Call:
mantel(xdis = RelatednessMat, ydis = DistMat, na.rm = TRUE) 

Mantel statistic r: -0.01699 
      Significance: 0.975 

Upper quantiles of permutations (null model):
   90%    95%  97.5%    99% 
0.0118 0.0151 0.0171 0.0216 
Permutation: free
Number of permutations: 999


In [101]:
mantel(RelatednessMat, DistMat, method="spearman", na.rm = TRUE)


Mantel statistic based on Spearman's rank correlation rho 

Call:
mantel(xdis = RelatednessMat, ydis = DistMat, method = "spearman",      na.rm = TRUE) 

Mantel statistic r: -0.00911 
      Significance: 0.901 

Upper quantiles of permutations (null model):
    90%     95%   97.5%     99% 
0.00928 0.01203 0.01389 0.01594 
Permutation: free
Number of permutations: 999


In [11]:
png("~/ClownfishGWAS/data/APCL/RelatednessAPCL_noL4437.png")
plot(UnqRelatedness[, phi] ~ UnqRelatedness[, dist_km], ylab="phi", xlab="distance (km)")
#abline(lm(phi ~ dist_km, data = UnqRelatedness), col = "blue")
dev.off()

png 
  2

In [21]:
head(UnqRelatedness[phi > 0.354])

INDV1,INDV2,N_AaAa,N_AAaa,N1_Aa,N2_Aa,RELATEDNESS_PHI,INDV1_lat,INDV1_lon,INDV2_lat,INDV2_lon,dist_km,phi
L0298,L0739,269,7,326,371,0.365854,10.86884,124.7165,10.87150,124.7143,0.3814991,0.365854
L0667,L0716,379,8,444,565,0.359762,10.83625,124.7466,10.87383,124.7109,5.7232561,0.359762
L0667,L0784,368,6,444,531,0.365128,10.83625,124.7466,10.87386,124.7106,5.7454588,0.365128
L0667,L0886,363,5,444,548,0.355847,10.83625,124.7466,10.87136,124.7144,5.2622646,0.355847
L0667,L0917,372,8,444,550,0.358149,10.83625,124.7466,10.86006,124.7224,3.7494040,0.358149
L0667,L1291,374,6,444,551,0.363819,10.83625,124.7466,10.87389,124.7105,5.7572591,0.363819


In [22]:
UnqRelatedness[INDV2 =="L0886" & INDV1 == "L0691"]

INDV1,INDV2,N_AaAa,N_AAaa,N1_Aa,N2_Aa,RELATEDNESS_PHI,INDV1_lat,INDV1_lon,INDV2_lat,INDV2_lon,dist_km,phi
L0691,L0886,367,0,377,548,0.396757,10.87152,124.7142,10.87136,124.7144,0.03185264,0.396757


In [35]:
summary(lm(phi ~ dist_km, data = UnqRelatedness))


Call:
lm(formula = phi ~ dist_km, data = UnqRelatedness)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.246741 -0.027743  0.000111  0.025838  0.270433 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.620e-01  6.481e-05  2499.2   <2e-16 ***
dist_km     -7.046e-05  3.871e-06   -18.2   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.04589 on 1147916 degrees of freedom
  (4520 observations deleted due to missingness)
Multiple R-squared:  0.0002886,	Adjusted R-squared:  0.0002877 
F-statistic: 331.3 on 1 and 1147916 DF,  p-value: < 2.2e-16


In [14]:
summary(lm(RELATEDNESS_PHI ~ dist_km, data = UnqRelatedness))


Call:
lm(formula = RELATEDNESS_PHI ~ dist_km, data = UnqRelatedness)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.75372 -0.02723  0.00071  0.02645  0.27117 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.612e-01  6.971e-05 2312.75   <2e-16 ***
dist_km     -6.299e-08  4.165e-09  -15.13   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.04938 on 1149432 degrees of freedom
  (4524 observations deleted due to missingness)
Multiple R-squared:  0.000199,	Adjusted R-squared:  0.0001981 
F-statistic: 228.8 on 1 and 1149432 DF,  p-value: < 2.2e-16


In [27]:
summary(lm(RELATEDNESS_PHI ~ dist_km, data = UnqRelatedness[INDV1 != "L4437" & INDV2 != "L4437"]))


Call:
lm(formula = RELATEDNESS_PHI ~ dist_km, data = UnqRelatedness[INDV1 != 
    "L4437" & INDV2 != "L4437"])

Residuals:
      Min        1Q    Median        3Q       Max 
-0.246741 -0.027743  0.000111  0.025838  0.270433 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.620e-01  6.481e-05  2499.2   <2e-16 ***
dist_km     -7.046e-05  3.871e-06   -18.2   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.04589 on 1147916 degrees of freedom
  (4520 observations deleted due to missingness)
Multiple R-squared:  0.0002886,	Adjusted R-squared:  0.0002877 
F-statistic: 331.3 on 1 and 1147916 DF,  p-value: < 2.2e-16


In [23]:
#who are the really negative individuals?
UnqRelatedness[phi < -0.4, .N, by=c("INDV1")][N >1 ]
UnqRelatedness[phi < -0.4, .N, by=c("INDV2")][N >1 ]

INDV1,N
L4437,23


INDV2,N
L4437,281


In [25]:
fish_meta[ligation_id =="L4437"]

fish_indiv,size,color,sex,gen_id,ligation_id,sample_id,site,date,anem_obs_time,gps,ligation_year,time_date,year,month,day,hour,minute,lat,lon
1956,1.4,BW,J,1956,L4437,APCL17_207,Elementary School,2017-05-23,12:06:10,5,2017,2017-05-23 04:06:10,2017,5,23,4,6,10.79952,124.7683
